# User_Query_Classifier

## 1. Model Setting - Gemini pro 1.0

In [1]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core

In [2]:
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

2024-05-07 07:00:39.146008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 07:00:40.208281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-07 07:00:40.208385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
vertexai.init(project="adsp-capstone-property-pilot", location="us-central1")

In [5]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

### Test

In [6]:
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hi there. 👋 How can I help you today?


## 2. Create the function for user query classification

In [7]:
def generate_prompt_classifier(instruction, user_query):
    return instruction.replace("{USER_QUERY}", user_query)

In [8]:
instruction_classifier = """
### Instructions ###
I want you to act as a sentence classifier. Based on "User's query" below, classify it into one of the categories listed, and respond with an integer from 0 to 2. 
Provide only the number as your answer. Do not include any other explanations in your response. Read the instructions below:

- If the question is about searching for rentals or real estate: output "1".
- If the question is about Chicago local information (e.g., neighborhoods, schools, restaurants, transportation): output "2".
- For any other questions: output "0".


Now, use the following query to determine the number.

### Example query ###
I want to buy a house in River North with at least two bedrooms.

### Example Output ###
1

### Example query ###
I am looking for a rental around Wicker Park in Chicago with a gym and a pool. It should be built within the last 10 years.

### Example Output ###
1


### Example query ###
What is the West Loop neighborhood in Chicago like?

### Example Output ###
2

### Example query ###
Could you recommend an Italian restaurant in Chicago?

### Example Output ###
2

### Example query ###
What is the nearest station to Lincoln Park?

### Example Output ###
2


### Example query ###
When was Llama 3 announced?

### Example Output ###
0

### Example query ###
How is Amazon's stock price doing?

### Example Output ###
0


### User’s query ###
{USER_QUERY}

### Output ###
"""

In [9]:
user_query = "I am looking for a two-bedroom."
print(generate_prompt_classifier(instruction_classifier, user_query))


### Instructions ###
I want you to act as a sentence classifier. Based on "User's query" below, classify it into one of the categories listed, and respond with an integer from 0 to 2. 
Provide only the number as your answer. Do not include any other explanations in your response. Read the instructions below:

- If the question is about searching for rentals or real estate: output "1".
- If the question is about Chicago local information (e.g., neighborhoods, schools, restaurants, transportation): output "2".
- For any other questions: output "0".


Now, use the following query to determine the number.

### Example query ###
I want to buy a house in River North with at least two bedrooms.

### Example Output ###
1

### Example query ###
I am looking for a rental around Wicker Park in Chicago with a gym and a pool. It should be built within the last 10 years.

### Example Output ###
1


### Example query ###
What is the West Loop neighborhood in Chicago like?

### Example Output ###
2



### Test

In [11]:
user_query = "I am looking for a two-bedroom."
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

1


In [12]:
user_query = "Are there any pet-friendly condos available for sale right now?"
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

1


In [13]:
user_query = "What are the top-rated elementary schools in Chicago's Lakeview neighborhood?"
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

2


In [14]:
user_query = "Can you recommend a good sushi restaurant near Millennium Park in Chicago?"
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

2


In [15]:
user_query = "What is the nearest station from The Loop?"
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

2


In [16]:
user_query = "Can you explain how a blockchain transaction works?"
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

0


In [17]:
user_query = "Hello!"
prompt = generate_prompt_classifier(instruction_classifier, user_query)
print(get_chat_response(chat, prompt))

0
